# Set

In [20]:
type Integer = int
type String = string
type Boolean = bool


type Shape = Circle | Triangle | Square

# Operation

In [21]:
type Operation<'T, 'U> = 'T -> 'U

let stringLength: Operation<String, Integer> = String.length
stringLength "foo"
|> printfn "stringLength: %O"

let integerParse: Operation<String, Integer> = Integer.Parse
integerParse "100"
|> printfn "integerParse: %O"

let divideBy: Operation<Integer, Integer -> Integer> = (/)
divideBy 4
|> printfn "divideBy: %O"


stringLength: 3
integerParse: 100
divideBy: Microsoft.FSharp.Core.OptimizedClosures+Invoke@3253[System.Int32,System.Int32,System.Int32]


# Algebraic structure

In [22]:
type AlgebraicStructure<'T, 'U>(operator: 'T -> 'U) = 
  member this.Apply = operator
  

let stringLengthAS = new AlgebraicStructure<String, Integer>(String.length)

stringLengthAS.Apply "foo"
|> printfn "stringGetLengthAS: %O"

stringGetLengthAS: 3


# Binary Operation

In [23]:
open System

type BinaryOperation<'T, 'U> = Operation<'T, 'T -> 'U>

let isEqualString: BinaryOperation<String, Boolean> = (=)
isEqualString "foo" "bar"
|> printfn "isEqualString: %O"

let split: BinaryOperation<String, String list> = fun a b -> 
  a.Split([|b|], StringSplitOptions.None) |> Array.toList
split "bar" "a"
|> printfn "split: %O"

isEqualString: False
split: [b; r]


# Closed operation

In [24]:
type ClosedOperation<'T> = Operation<'T, 'T>

let reverseList: ClosedOperation<List<'T>> = List.rev
reverseList [Circle; Triangle; Square]
|> printfn "reverseList: %O"

let negate: ClosedOperation<'T -> Boolean> =
  fun f a -> f a |> not
(negate (fun _ -> true))()
|> printfn "negate: %O"

reverseList: [Square; Triangle; Circle]
negate: False


# Closed binary operation

In [25]:
type ClosedBinaryOperation<'T> = BinaryOperation<'T, 'T>
// type ClosedBinaryOperation<'T> = Operation<'T, ClosedOperation<'T>>

let subtract: ClosedBinaryOperation<Integer> = (-)
subtract 3 2
|> printfn "subtract: %O"

let intersect: ClosedBinaryOperation<Set<Shape>> = Set.intersect
intersect
  (Set.ofList [Circle; Triangle])
  (Set.ofList [Triangle; Square])
|> printfn "subtract: %O"

subtract: 1
subtract: set [Triangle]



# Magma

In [26]:
type Magma<'T>(combine: ClosedBinaryOperation<'T>) = 
  member this.Combine = combine

let sumMagma = new Magma<Integer>((+))
sumMagma.Combine 1 2
|> printfn "sumMagma: %O"

let composeMagma = new Magma<Integer -> Integer>((>>))
composeMagma.Combine ((+) 1) ((-) 3)
|> fun f -> f 1
|> printfn "composeMagma: %O"

sumMagma: 3
composeMagma: 1


# Associative property

In [27]:
(=)
  ((1 + 2) + 3)
  (1 + (2 + 3))
|> printfn "+: %O"

(=)
  ((1 - 2) - 3)
  (1 - (2 - 3))
|> printfn "-: %O"

(=)
  (([Circle; Triangle] @ [Square; Circle]) @ [Triangle; Square])
  ([Circle; Triangle] @ ([Square; Circle] @ [Triangle; Square]))
|> printfn "@: %O"

+: True
-: False
@: True


# Semigroup

In [28]:
type Semigroup<'T>(combine) =
  inherit Magma<'T>(combine)
  member this.Reduce (values: 'T list) =
      List.reduce this.Combine values

let andSemiGroup = new Semigroup<Boolean>((&&))
andSemiGroup.Reduce [true; true; true; false]
|> printfn "andSemiGroup: %O"

let intersectSemiGroup = new Semigroup<Set<Shape>>(Set.intersect)
intersectSemiGroup.Reduce 
  ([
    Set.ofList [Circle; Triangle;];
    Set.ofList [Square; Circle;];
    Set.ofList [Circle; Circle;]
  ])
|> printfn "intersectSemiGroup: %O"

andSemiGroup: False
intersectSemiGroup: set [Circle]


# Identity element

In [29]:
(
  10 + 0 = 10,
  0 + 10 = 10
)
|> fun (a, b) -> a && b
|> printfn "(+, 0): %O"

(
  (false && true) = false,
  (true && false) = false
)
|> fun (a, b) -> a && b
|> printfn "(&&, true): %O"

(
  [Circle] @ [] = [Circle],
  [] @ [Circle] = [Circle]
)
|> fun (a, b) -> a && b
|> printfn "(@, []): %O"

(+, 0): True
(&&, true): True
(@, []): True


# Monoid
[A Table of classification](https://fsharpforfunandprofit.com/posts/monoids-without-tears/#a-table-of-classifications)

In [30]:
type Monoid<'T>(combine, empty) =
  inherit Semigroup<'T>(combine)
  member this.Empty: 'T = empty
  member this.Reduce (values: 'T list) =
      List.fold this.Combine this.Empty values

let multiplyMonoid = new Monoid<Integer>(( * ), 1)
multiplyMonoid.Reduce [3; 4; 5]
|> printfn "multiplyMonoid: %O"

let maxMonoid = new Monoid<Integer>(max, Integer.MinValue)
maxMonoid.Reduce [129; 123; 92]
|> printfn "maxMonoid: %O"

multiplyMonoid: 60
maxMonoid: 129


## Parallelization

In [31]:
open System.Linq

type ParallelMonoid<'T>(combine, empty) =
  inherit Monoid<'T>(combine, empty)
  member this.Reduce (values: 'T list) =
      values.AsParallel().Aggregate(this.Empty, this.Combine)

let addMonoid = new ParallelMonoid<Integer>((+), 0);
[1..100]
|> addMonoid.Reduce
|> printfn "addMonoid: %O"

addMonoid: 5050


# Incrementalism

In [32]:
let t1 = maxMonoid.Reduce [1; 2; 3;]
printfn "T1: %O" t1

maxMonoid.Reduce [t1; 4]
|> printfn "T2: %O"

T1: 3
T2: 4
